Раздел: Начальные действия.
Пункт 4: Разработать и применить генератор тестовых данных.

In [34]:
import datetime
import random
import pyodbc
from faker import Faker

In [35]:
database = 'PaymentData'
user = 'sa'
password = 'reallyStrongPwd123'
cnxn = pyodbc.connect('DSN='+database+';DATABASE='+database+';UID='+user+';PWD='+ password)
cursor = cnxn.cursor()

faker = Faker()

num_of_lines = 1

Заполняем PaymentParticipant:

In [36]:
def gener_date(start_time, end_time):
    return datetime.datetime.fromtimestamp(
            random.randint(int(start_time.timestamp()), int(end_time.timestamp()))
        )

In [37]:
for _ in range(num_of_lines):
    balance = 0
    balance2 = 0
    balance3 = 0
    name = faker.name()
    optimistic_lock_field = random.randint(1, 3)
    object_type = random.randint(0, 4)
    bank_details = faker.bban()
    active_from = gener_date(datetime.datetime(2010, 1, 1), datetime.datetime.now())
    inactive_from = gener_date(active_from, datetime.datetime.now())
    active_from = active_from.isoformat()
    inactive_from = inactive_from.isoformat()

    query = f"""INSERT INTO [dbo].[PaymentParticipant] ([Oid], [Balance], [Name], [OptimisticLockField], [GCRecord], 
                [ObjectType], [ActiveFrom], [InactiveFrom], [BankDetails], [Balance2], [Balance3]) 
                VALUES (NEWID(), {balance}, '{name}', {optimistic_lock_field}, Null, {object_type}, '{active_from}',
                '{inactive_from}', '{bank_details}', {balance2}, {balance3})"""

    cursor.execute(query)
    cursor.commit()

In [40]:
#example
query = "SELECT * FROM [dbo].[PaymentParticipant]"
cursor.execute(query)
for row in cursor.fetchall():
    print(row)

('49931082-2B9F-4B83-8999-552D11310006', 0, 'David Michael', 2, None, 2, datetime.datetime(2017, 4, 20, 21, 57, 27), datetime.datetime(2018, 2, 14, 12, 1, 28), 'GMQC15935104195503', 0, 0)


In [41]:
query = "DELETE FROM [dbo].[PaymentParticipant]"
cursor.execute(query)
cursor.commit()

In [42]:
num_of_lines = 1000
for _ in range(num_of_lines):
    balance = 0
    balance2 = 0
    balance3 = 0
    name = faker.name()
    optimistic_lock_field = random.randint(1, 3)
    object_type = random.randint(0, 4)
    bank_details = faker.bban()
    active_from = gener_date(datetime.datetime(2010, 1, 1), datetime.datetime.now())
    inactive_from = gener_date(active_from, datetime.datetime.now())
    active_from = active_from.isoformat()
    inactive_from = inactive_from.isoformat()

    query = f"""INSERT INTO [dbo].[PaymentParticipant] ([Oid], [Balance], [Name], [OptimisticLockField], [GCRecord], 
                [ObjectType], [ActiveFrom], [InactiveFrom], [BankDetails], [Balance2], [Balance3]) 
                VALUES (NEWID(), {balance}, '{name}', {optimistic_lock_field}, Null, {object_type}, '{active_from}',
                '{inactive_from}', '{bank_details}', {balance2}, {balance3})"""

    cursor.execute(query)
    cursor.commit()

Заполняем Supplier:
(PS: пусть object_type supplier'a = 0, object_type employer'a = 1, object_type client'a = 2, object_type cashbox'a = 3, object_type bank'a = 4)

In [43]:
supplier_obj_type = 0
query = f"SELECT Oid, Name FROM [dbo].[PaymentParticipant] WHERE ObjectType = {supplier_obj_type}"
participants = cursor.execute(query)
for participant in participants.fetchall():
    oid, contact = participant
    query = f"""INSERT INTO [dbo].[Supplier] ([Oid], [Contact], [ProfitByMaterialAsPayer], [ProfitByMaterialAsPayee], [CostByMaterialAsPayer])
                VALUES ('{oid}', '{contact}', {random.randint(0, 1)}, {random.randint(0, 1)}, {random.randint(0, 1)})"""
    cursor.execute(query)
cursor.commit()

In [47]:
#example
query = "SELECT TOP (1) * FROM [dbo].[Supplier]"
cursor.execute(query)
for row in cursor.fetchall():
    print(row)

('08674F50-62EC-411B-A2B0-04870B85C712', 'Christine Fowler', True, True, False)


Заполняем Employee:

In [48]:
employee_obj_type = 1
query = f"SELECT Oid, Name FROM [dbo].[PaymentParticipant] WHERE ObjectType = {employee_obj_type}"
participants = cursor.execute(query)
for participant in participants.fetchall():
    oid, name = participant
    first_name, second_name = name.split(maxsplit=1)
    busy_until = gener_date(datetime.datetime(2010, 1, 1), datetime.datetime.now()).isoformat()
    stuff = random.randint(0, 100000)
    hour_price = random.randint(100, 2000)
    plan_fix_id = random.randint(1, 10000)
    plan_fix_money_request_task = faker.text(100)

    query = f"""INSERT INTO [dbo].[Employee] ([Oid], [BusyUntil], [SecondName], [Stuff], [HourPrice], [Patronymic], 
                [PlanfixId], [Head], [PlanfixMoneyRequestTask])
                VALUES ('{oid}', '{busy_until}', '{second_name}', {stuff}, {hour_price}, '{first_name}',
                        {plan_fix_id}, Null, '{plan_fix_money_request_task}')"""
    cursor.execute(query)
cursor.commit()

In [49]:
#example
query = "SELECT TOP (1) * FROM [dbo].[Employee]"
cursor.execute(query)
for row in cursor.fetchall():
    print(row)

('19FD7311-1D40-4720-B4C4-016DF499CE3F', datetime.datetime(2011, 8, 21, 15, 19, 55), 'Best', 58893, 1386, 'Latoya', 9478, None, 'Record education war energy near others Democrat power.')


Заполняем Client:

In [50]:
client_obj_type = 2
query = f"SELECT Oid, Name FROM [dbo].[PaymentParticipant] WHERE ObjectType = {client_obj_type}"
participants = cursor.execute(query)
for participant in participants.fetchall():
    oid, name = participant
    first_name, second_name = name.split(maxsplit=1)
    phone_number = faker.phone_number()
    query = f"""INSERT INTO [dbo].[Client] ([Oid], [FirstName], [SecondName], [Phone])
                VALUES ('{oid}', '{first_name}', '{second_name}', '{phone_number}')"""
    cursor.execute(query)
cursor.commit()

In [51]:
#example
query = "SELECT TOP (1) * FROM [dbo].[Client]"
cursor.execute(query)
for row in cursor.fetchall():
    print(row)

('70FBB4DF-FC02-4544-8C35-00AF24E989B7', 'Laura', 'Schneider', '(601)736-6274x022')


Заполняем Project:

In [55]:
oid_employee = []
query = "SELECT Oid FROM [dbo].[Employee]"
participants = cursor.execute(query)
for participant in participants:
    oid_employee.append(participant[0])

oid_client = []
query = "SELECT Oid FROM [dbo].[Client]"
participants = cursor.execute(query)
for participant in participants:
    oid_client.append(participant[0])

for _ in range(num_of_lines):
    address = faker.address()
    area = random.randint(1, 1000000)
    balance = 0
    balance_by_material = 0
    balance_by_work = 0
    client = random.choice(oid_client)
    start_date = gener_date(datetime.datetime(2010, 1, 1), datetime.datetime.now())
    finish_date = gener_date(start_date + datetime.timedelta(180),start_date + datetime.timedelta(3650),)
    manager, foreman = random.sample(oid_employee, 2)
    name = faker.bs()
    optimistic_lock_field = random.randint(1, 3)
    plan_fix_change_request_task = faker.text(100)
    plan_fix_work_task = faker.text(100)
    remainder_the_advance = 0
    status = random.randint(0, 4)
    use_analytics = random.randint(0, 1)
    work_price_rate = random.uniform(100, 100000000)
    workers_price_rate = random.uniform(100, 500000)

    query = f"""INSERT INTO [dbo].[Project] ([Oid], [Name], [Address], [Client], [Manager], [Foreman], [OptimisticLockField], [GCRecord], 
                [Balance], [BalanceByMaterial], [BalanceByWork], [PlaningStartDate], [Status], [FinishDate], [Area], [WorkPriceRate],
                [WorkersPriceRate], [RemainderTheAdvance], [PlanfixWorkTask], [PlanfixChangeRequestTask], [UseAnalytics]) 
                VALUES (NEWID(), '{name}', '{address}', '{client}', '{manager}', '{foreman}', {optimistic_lock_field}, 
                        Null, {balance}, {balance_by_material}, {balance_by_work}, '{start_date}', {status}, '{finish_date}', 
                        {area}, {work_price_rate}, {workers_price_rate}, {remainder_the_advance}, '{plan_fix_work_task}', 
                        '{plan_fix_change_request_task}', {use_analytics})"""
    cursor.execute(query)
cursor.commit()



In [56]:
#example
query = "SELECT TOP (1) * FROM [dbo].[Project]"
cursor.execute(query)
for row in cursor.fetchall():
    print(row)

('23B9ED2E-FBBB-4904-B9C6-00063B8D45B2', 're-contextualize world-class mindshare', '386 Victor Burg Apt. 059\nNorth Branditown, NV 15692', 'C8064AF2-9EFE-4914-997A-2BAAE00DE4B4', '71F76812-2E02-4CE4-A6DE-3DD5F6F5C5C2', 'E6B2DC1D-1CED-4B4E-84AB-76EF7D9E6AB1', 3, None, 0, 0, 0, datetime.datetime(2015, 4, 16, 21, 27, 8), 0, datetime.datetime(2017, 1, 10, 0, 45, 46), 512564, 13061389.633202966, 360646.26448074606, 0, 'Can expect truth together summer heart.', 'Tough yard until power particular message. Live hit arm claim hospital her major.', False)


Заполняем Cashbox:

In [57]:
cashbox_obj_type = 3
oid_acc_type = []
query = "SELECT Oid FROM [dbo].[AccountType]"
participants = cursor.execute(query)
for participant in participants:
    oid_acc_type.append(participant[0])

query = f"SELECT Oid FROM [dbo].[PaymentParticipant] WHERE ObjectType = {cashbox_obj_type}"
participants = cursor.execute(query)
for participant in participants.fetchall():
    account_type = random.choice(oid_acc_type)
    query = f"""INSERT INTO [dbo].[Cashbox] ([Oid], [AccountType]) VALUES ('{participant[0]}', '{account_type}')"""
    cursor.execute(query)
cursor.commit()

In [58]:
#example
query = "SELECT TOP (1) * FROM [dbo].[Cashbox]"
cursor.execute(query)
for row in cursor.fetchall():
    print(row)

('B9681C10-2D44-4D17-8B13-01E6450616B5', 'C14044C4-0D50-4BCE-AF32-0639F7738026')


Заполняем Bank:

In [59]:
bank_obj_type = 4
oid_acc_type = []
query = "SELECT Oid FROM [dbo].[AccountType]"
participants = cursor.execute(query)
for participant in participants:
    oid_acc_type.append(participant[0])

query = f"SELECT Oid FROM [dbo].[PaymentParticipant] WHERE ObjectType = {bank_obj_type}"
participants = cursor.execute(query)
for participant in participants.fetchall():
    account_type = random.choice(oid_acc_type)
    oid = participant[0]
    query = f"""INSERT INTO [dbo].[Bank] ([Oid], [AccountType]) VALUES ('{oid}', '{account_type}')"""
    cursor.execute(query)
cursor.commit()

In [60]:
#example
query = "SELECT TOP (1) * FROM [dbo].[Bank]"
cursor.execute(query)
for row in cursor.fetchall():
    print(row)

('9FCE29AE-E0DC-45C8-BD44-03BC90A2C479', 'C14044C4-0D50-4BCE-AF32-0639F7738026')


Заполняем Payment:

In [62]:
oid_category = []
query = "SELECT Oid FROM [dbo].[PaymentCategory]"
participants = cursor.execute(query)
for participant in participants:
    oid_category.append(participant[0])

oid_participant = []
query = "SELECT Oid FROM [dbo].[PaymentParticipant]"
participants = cursor.execute(query)
for participant in participants:
    oid_participant.append(participant[0])

oid_project = []
query = "SELECT Oid FROM [dbo].[Project]"
participants = cursor.execute(query)
for participant in participants:
    oid_project.append(participant[0])

for _ in range(num_of_lines):
    amount = random.randint(1, 100000)
    category = random.choice(oid_category)
    check_number = random.randint(10000, 99999)
    comment = faker.text(100)
    date = gener_date(datetime.datetime(2010, 1, 1), datetime.datetime.now()).isoformat()
    created_date = datetime.datetime.fromtimestamp(
        int(datetime.datetime.now().timestamp())
    ).isoformat()
    is_authorized = random.randint(0, 1)
    justificaton = faker.text(30)
    number = random.randint(10000, 99999)
    optimistic_lock_field = random.randint(1, 3)
    payee, payer = random.sample(oid_participant, 2)
    project = random.choice(oid_project)

    query = f"""INSERT INTO [dbo].[Payment] ([Oid], [Amount], [Category], [Project], [Justification], [Comment], [Date], [Payer], [Payee],
                [OptimisticLockField], [GCRecord], [CreateDate], [CheckNumber], [IsAuthorized], [Number]) 
                VALUES (NEWID(), {amount}, '{category}', '{project}', '{justificaton}', '{comment}', '{date}', '{payer}', '{payee}', 
                {optimistic_lock_field}, Null, '{created_date}', '{check_number}', {is_authorized}, '{number}')"""
    cursor.execute(query)
cursor.commit()

In [63]:
#example
query = "SELECT TOP (1) * FROM [dbo].[Payment]"
cursor.execute(query)
for row in cursor.fetchall():
    print(row)

('AC47AD04-70D9-436A-B07F-005618FE4FC0', 77604, 'CDCA2762-818E-408A-9E57-08C515C2D87E', 'CB815C04-9FF9-4D63-B384-D004D9D77DEC', 'Seat produce wide blue PM.', 'Every evidence ball computer poor. Significant car different about stop talk.', datetime.datetime(2014, 4, 5, 7, 10, 7), '750C22A9-0B5A-4997-B8BC-539458431992', '902BCF08-46A1-4F8A-B7D2-8AF08E689166', 2, None, datetime.datetime(2022, 6, 11, 20, 52, 46), '49883', True, '27256')
